This notbook maps the features from each dataset from protein families to GO terms using the UniProt web API

In [1]:
import pickle
import pandas as pd
import numpy as np
import requests
import concurrent.futures as cf
from tqdm import tqdm
import os
import wget
from Bio import SeqIO

data_path = '../data/'

DNA Δ Features

In [2]:
df_DNA_delta = None
with open('../data/OriginalDataframes/DNA_diff_std.pkl', 'rb') as file:
    df_DNA_delta = pickle.load(file)
DNA_delta_columns = [col for col in df_DNA_delta.columns.values if col != 'diagnosis']
len(DNA_delta_columns)

56212

RNA Δ Features

In [3]:
df_RNA_delta = None
with open('../data/OriginalDataframes/RNA_diff_std.pkl', 'rb') as file:
    df_RNA_delta = pickle.load(file)
RNA_delta_columns = [col for col in df_RNA_delta.columns.values if col != 'diagnosis']
len(RNA_delta_columns)

18693

DNA % Features

In [4]:
df_DNA_filter = None
with open('../data/OriginalDataframes/DNA_filter_std.pkl', 'rb') as file:
    df_DNA_filter = pickle.load(file)
DNA_filter_columns = [col for col in df_DNA_filter.columns.values if col != 'diagnosis']
len(DNA_filter_columns)

15130

RNA % Features

In [5]:
df_RNA_filter = None
with open('../data/OriginalDataframes/RNA_filter_std.pkl', 'rb') as file:
    df_RNA_filter = pickle.load(file)
RNA_filter_columns = [col for col in df_RNA_filter.columns.values if col != 'diagnosis']
len(RNA_filter_columns)

7655

In [6]:
full_list = DNA_delta_columns + RNA_delta_columns + DNA_filter_columns + RNA_filter_columns
feature_set = set(full_list)
print(f'Total features: {len(full_list)}')
print(f'Unique Features: {len(feature_set)}')
print(f'Difference: {len(full_list) - len(feature_set)}')

Total features: 97690
Unique Features: 60181
Difference: 37509


In [7]:
gene_set = [feature.split('_')[1] for feature in feature_set]
print(gene_set[:3])

['Q64MK3', 'H1CJ82', 'R5SPN9']


In [8]:
def download_gene(gene):
    if f'{gene}.txt' not in uniprot_files:
        try:
            wget.download(f'https://www.uniprot.org/uniprot/{gene}.txt', uniprot_path)
        except:
            pass

In [9]:
uniprot_path = data_path + 'uniprot_files'
uniprot_files = os.listdir(uniprot_path)
with cf.ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(download_gene, gene_set), total = len(gene_set)))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60181/60181 [00:06<00:00, 8942.79it/s]


Uniprot API yields SwissProt format files:
https://web.expasy.org/docs/userman.html

In [10]:
uniprot_files = os.listdir(uniprot_path)
len(uniprot_files)

59871

In [11]:
def check_file_content(gene):
    if os.stat(uniprot_path+ '/' + gene).st_size > 0:
        return gene
    else:
        False

In [12]:
good_files = []
with tqdm(total=len(uniprot_files), desc='Finding Full Files: ') as pbar:
    with cf.ThreadPoolExecutor() as executor:
        futures = [executor.submit(check_file_content, file) for file in uniprot_files]
        for future in cf.as_completed(futures):
            res = future.result()
            if res:
                good_files.append(res)
            pbar.update(1)
print(len(good_files))

Finding Full Files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 59871/59871 [00:04<00:00, 12658.85it/s]

52002


In [13]:
def extract_GO(gene):
    path = uniprot_path + '/' + gene
    record = None
    for rec in SeqIO.parse(path, 'swiss'):
        record = rec
    return gene[:-4], [xref for xref in record.dbxrefs if xref.startswith('GO:')]

In [17]:
GO_map = {}
with tqdm(total=len(good_files), desc='Mapping GO Terms: ') as pbar:
    with cf.ThreadPoolExecutor() as executor:
        futures = [executor.submit(extract_GO, file) for file in good_files]
        for future in cf.as_completed(futures):
            gene, GOs = future.result()
            if GOs:
                GO_map[gene] = GOs
            pbar.update(1)

Mapping GO Terms: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52002/52002 [02:47<00:00, 310.72it/s]


len(GO_map) = 34402

strip dbxref from tag: GO:GO:tag -> GO:tag

In [26]:
same_prefix = True
for gene in GO_map:
    GO_map[gene] = [tag[3:] for tag in GO_map[gene]]

In [25]:
with open('../data/GO_map.pkl', 'wb') as handle:
    pickle.dump(GO_map, handle, protocol=pickle.HIGHEST_PROTOCOL)